In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.stats.api as sms
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_table("http://www.econometrics.com/comdata/gujarati/data_15.7.shd", header = None,
                    delim_whitespace = True,
                    names = ["Fumador","Edad","Escolaridad","Ingresos","Pcigs79"])

In [ ]:
grafico = sns._____(______)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax._____(data["______"])
plt.show()

### MPL

In [ ]:
Y = data["______"]
X = data[["_____","_____","_____","_____"]]

In [ ]:
MLP = sm.OLS(____, sm.____(____))
resultados_MLP = ____.____
print(resultados_MLP.summary())

In [ ]:
MLP_marg_eff = ________.________ 

In [ ]:
prediccion_MLP = ________.________
prediccion_MLP = pd.________(________, columns = ["Prediccion MLP"])

### Modelo Logit

In [ ]:
modelo_logit = sm.____(____, sm.____(____))

In [ ]:
logit_marg_eff = __________.__________
print(__________.__________)

In [ ]:
logit_marg_eff = __________.__________(__ = "__________")
print(__________.__________)

**Ejercicio modelo LOGIT con escolaridad únicamente de más de 10 años**

In [ ]:
data_ejercicio = _____________

### Modelo Probit

In [ ]:
modelo_probit = sm.___(___,sm.___(___))
resultados_probit = __________.__________
print(__________.__________)

In [ ]:
probit_mar_eff = __________.__________
print(__________.__________)

In [ ]:
probit_mar_eff = __________.__________(__ = "__________")
print(__________.__________)

**Ejercicio modelo PROBIT con escolaridad únicamente de más de 10 años**

In [ ]:
from statsmodels.iolib.summary2 import summary_col

In [ ]:
tabla = __________(results=[__________,__________,__________],
                   model_names = ["MLP","Logit","Probit"], stars= __________)
print(tabla)

In [ ]:
coeficientes = ["Edad","Escolaridad","Ingresos","Pcigs79"]
__________ = pd.__________(__________.__________,
                           index= coeficientes,
                           columns = ["Logit"])

__________ = pd.__________(__________.__________,
                           index= coeficientes,
                           columns = ["Probit"])

__________ = pd.__________(__________[1:5],
                         columns = ["MLP"])

In [ ]:
pd.concat([__________, __________, __________], ____ = _)